# Benchmarking
This Notebook allows you to benchmark the different models.

In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
from utils import init_notebook, benchmarking, predictions
%aimport datasets, utils.predictions, utils.benchmarking,utils.data_cleaning
from utils.predictions import * 

import sahi
from sahi import AutoDetectionModel#, get_prediction
import fiftyone as fo
import torch

import os
from pathlib import Path

HOME = Path(os.getcwd()).parents[0]
HOME

print("torch available :",torch.cuda.is_available())
torch.cuda.empty_cache()

torch available : True


## Model Importation
For the benchmarking, we decide to import all the models using SAHI so that the objects handled are all sahi Autodetection models

### YOLO

In [3]:
model_name="yolass_aug_26ep"

yolo_detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",    # Model type (base model is yolov8 also for yolov10)
    model_path=f"../outputs/yolo/{model_name}.pt",    # Path to the model weights
    confidence_threshold=0.1,   # Confidence threshold
    # The higher the confidence threshold, the more precise they are (but we do a filtering later)
    device="cuda:0",  # to use the GPU
)

### DETR

In [28]:
from transformers import DetrForObjectDetection,DetrImageProcessor

# settings
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIDENCE_TRESHOLD = 0.1
IOU_TRESHOLD = 0.7

model_path=HOME/"outputs/detr/detr_cocass/detr_cocass_v2"

CHECKPOINT = 'facebook/detr-resnet-101'

image_processor = DetrImageProcessor.from_pretrained(CHECKPOINT)
model=DetrForObjectDetection.from_pretrained(model_path)    # load the model from the checkpoint as a DEtrForObjectDetection model pretrained by the weights of the model_path
#model = Detr(model)
model.to(DEVICE)

09/04/2024 13:32:55 - INFO - timm.models._builder -   Loading pretrained weights from Hugging Face hub (timm/resnet101.a1h_in1k)
09/04/2024 13:32:55 - INFO - timm.models._hub -   [timm/resnet101.a1h_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
09/04/2024 13:32:55 - INFO - timm.models._builder -   Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.


DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [30]:
detr_detection_model = AutoDetectionModel.from_pretrained(
    model_type="huggingface",
    #config_path="../../resources/models/detr/detr_coco7_2800/config.json",
    model=model,
    #model_path=model_path,
    processor=image_processor,
    confidence_threshold=0.7,
    device="cuda:0", # or 'cuda:0'
)
type(detr_detection_model)

sahi.models.huggingface.HuggingfaceDetectionModel

### Detectron2

In [31]:
model_path = "../outputs/detectron2/faster_rcnn_v3.pth"
det2_detection_model = AutoDetectionModel.from_pretrained(
    model_type='detectron2',
    #model=predictor,
    model_path=model_path,
    config_path="../models/detectron2/config4.yaml",
    #category_remapping= {i: class_name for i, class_name in enumerate(dataset.get_classes('annotations'))},
    confidence_threshold=0.1,
    image_size=1280,
    device="cuda:0", # or 'cuda:0'
)

../models/detectron2/config4.yaml not available in Model Zoo!


09/04/2024 13:33:07 - INFO - detectron2.checkpoint.detection_checkpoint -   [DetectionCheckpointer] Loading from ../outputs/detectron2/faster_rcnn_v3.pth ...
09/04/2024 13:33:07 - INFO - fvcore.common.checkpoint -   [Checkpointer] Loading from ../outputs/detectron2/faster_rcnn_v3.pth ...
09/04/2024 13:33:07 - WARNING - sahi.models.detectron2 -   Attribute 'thing_classes' does not exist in the metadata of dataset 'my_dataset_train': metadata is empty.


## Test Dataset

We use fiftyone to iterate over the samples of the test dataset

In [12]:
# Importation of the dataset
dataset = fo.Dataset.from_dir(
    data_path=HOME/"data/coco_datasets/Cocass_aug/images",  # path to the dataset
    labels_path=HOME/"data/coco_datasets/Cocass_aug/fraw_detailed_val.json",   # path to the labels/annotations file
    dataset_type=fo.types.COCODetectionDataset, # the type of dataset to import
)
dataset

# the annotations are imported as "detections" but we want to rename them as "annotations" to avoid confusion
dataset.rename_sample_field("detections", "annotations")

 100% |█████████████████████| 0/0 [741.9us elapsed, ? remaining, ? samples/s] 


09/03/2024 13:56:29 - INFO - eta.core.utils -    100% |█████████████████████| 0/0 [741.9us elapsed, ? remaining, ? samples/s] 


AttributeError: Sample field 'detections' does not exist

In [203]:
# To inspet the dataset on an external app
session = fo.launch_app(dataset, auto=False)
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


08/30/2024 10:33:14 - INFO - fiftyone.core.session.session -   Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

If the the dataset imported is made of crops (if not skip this kernel)

In [33]:
remapping={i: class_name for i, class_name in enumerate(dataset.get_classes('annotations'))}    # remapping of the labels for faster R-CNN (sometime the label are not kept into the configs)
label_set ={'0', '1', '10', '11', '13','14', '15', '2', '3', '4', '5', '6', '7', '8', '9'}
kwargs={"slice_height":1280, "slice_width":1280, "overlap_height_ratio":0.9, "overlap_width_ratio":0.9}

for sample in dataset.iter_samples(progress=True, autosave=True):
    yolo_results=fo_predict_simple(yolo_detection_model,sample, 
        label_field="yolo_predictions", 
        kwargs=kwargs)
    # detr_results=fo_predict_simple(detr_detection_model,sample, 
    #     label_field="detr_predictions", 
    #     kwargs=kwargs)      
    # fo_predict_simple(det2_detection_model,sample, 
    #     label_field="det2_predictions", 
    #     kwargs=kwargs)
    # for det in sample["det2_predictions"].detections:
    #     det.label = remapping[int(det.label)]    

 100% |█████████████████| 997/997 [5.0m elapsed, 0s remaining, 3.1 samples/s]      


09/04/2024 11:22:33 - INFO - eta.core.utils -    100% |█████████████████| 997/997 [5.0m elapsed, 0s remaining, 3.1 samples/s]      


If the dataset is only a full page

In [ ]:
from utils.predictions import fo_predict_with_slicing
remapping={i: class_name for i, class_name in enumerate(dataset.get_classes('annotations'))}
label_set ={'0', '1', '10', '11', '13','14', '15', '2', '3', '4', '5', '6', '7', '8', '9'}
kwargs={"slice_height":1280, "slice_width":1280, "overlap_height_ratio":0.9, "overlap_width_ratio":0.9}
# We use Sliced Aided Hyper Inference to predict on the dataset
for sample in dataset.iter_samples(progress=True, autosave=True):
    yolo_results=fo_predict_with_slicing(yolo_detection_model,sample, 
        label_field="yolo_predictions", 
        slice_height=640, slice_width=640, overlap_height_ratio=0.8, overlap_width_ratio=0.8)
    detr_results=fo_predict_with_slicing(detr_detection_model,sample, 
        label_field="detr_predictions", 
        slice_height=640, slice_width=640, overlap_height_ratio=0.8, overlap_width_ratio=0.8)      
    fo_predict_with_slicing(det2_detection_model,sample, 
        label_field="det2_predictions", 
        slice_height=640, slice_width=640, overlap_height_ratio=0.8, overlap_width_ratio=0.8)
    for det in sample["det2_predictions"].detections:
        det.label = remapping[int(det.label)]   

This part as been added to export the results in COCO format in json to filter them and return them on fiftyone

In [34]:
from utils.data_cleaning import fiftyone_extraction_remapping,filter_annotations
import fiftyone.utils.coco as fouc
import json

# YOLO predictions filtering
# ------------------------
# Export the dataset to a JSON file
gt_json = HOME/"data/coco_datasets/Cocass_aug/fraw_detailed_val.json"
gt_json = HOME/"data/coco_datasets/tests/Cassini_009_LoC/f009_detailed_updated.json"
dataset.export(
    export_dir="results/jsons/f9/yolo_predictions",
    data_path=HOME/"data/coco_datasets/Cocass_aug/images",
    dataset_type=fo.types.COCODetectionDataset,  # You can choose other formats as well
    label_field="yolo_predictions",
    classes=dataset.get_classes('annotations'),
    export_media=False  # Set to True if you want to export media files as well
)
fiftyone_extraction_remapping(HOME/"notebooks/results/jsons/f9/yolo_predictions/labels.json",
                              gt_json,inplace=True)
filter_annotations(json_file_path=HOME/"notebooks/results/jsons/f9/yolo_predictions/labels.json",conf_threshold=0.5,
                    height_width_ratio= 0.3,
                    second_height_width_ratio=3,
                    iou_threshold= 0.8,inside_threshold=0.2,inplace=False,keep_annotations=True,class_agnostic=False)
fouc.add_coco_labels(
    dataset,
    "yolo_predictions_filtered",
    (HOME/"notebooks/results/jsons/f9/yolo_predictions/filtered_labels.json").as_posix(),
    classes=dataset.get_classes('annotations'),
)

# DETR predictions filtering
# ------------------------
dataset.export(
    export_dir="results/jsons/f9/detr_predictions",
    dataset_type=fo.types.COCODetectionDataset,  # You can choose other formats as well
    label_field="detr_predictions",
    # classes=dataset.get_classes('annotations'),
    export_media=False  # Set to True if you want to export media files as well
)
fiftyone_extraction_remapping(HOME/"notebooks/results/jsons/f9/detr_predictions/labels.json",
                              gt_json,inplace=True)
filter_annotations(json_file_path=HOME/"notebooks/results/jsons/f9/detr_predictions/labels.json",conf_threshold=0.5,
                    height_width_ratio= 0.3,
                    second_height_width_ratio=3,
                    iou_threshold= 0.8,inside_threshold=0.2,inplace=False,keep_annotations=True,class_agnostic=False)
fouc.add_coco_labels(
    dataset,
    "detr_predictions_filtered",
    (HOME/"notebooks/results/jsons/f9/detr_predictions/filtered_labels.json").as_posix(),
    classes=dataset.get_classes('annotations'),
)

# DET2 predictions filtering
# ------------------------
dataset.export(
    export_dir="results/jsons/f9/det2_predictions",
    dataset_type=fo.types.COCODetectionDataset,  # You can choose other formats as well
    label_field="det2_predictions",
    # classes=dataset.get_classes('annotations'),
    export_media=False  # Set to True if you want to export media files as well
)
fiftyone_extraction_remapping(HOME/"notebooks/results/jsons/f9/det2_predictions/labels.json",
                              gt_json,inplace=True)
filter_annotations(json_file_path=HOME/"notebooks/results/jsons/f9/det2_predictions/labels.json",conf_threshold=0.5,
                    height_width_ratio= 0.3,
                    second_height_width_ratio=3,
                    iou_threshold= 0.8,inside_threshold=0.2,inplace=False,keep_annotations=True,class_agnostic=False)
fouc.add_coco_labels(
    dataset,
    "det2_predictions_filtered",
    (HOME/"notebooks/results/jsons/f9/det2_predictions/filtered_labels.json").as_posix(),
    classes=dataset.get_classes('annotations'),
)

Directory 'results/jsons/f9/yolo_predictions' already exists; export will be merged with existing files


09/04/2024 11:22:33 - WARNING - fiftyone.core.collections -   Directory 'results/jsons/f9/yolo_predictions' already exists; export will be merged with existing files


 100% |█████████████████| 997/997 [7.2s elapsed, 0s remaining, 73.1 samples/s]       


09/04/2024 11:22:40 - INFO - eta.core.utils -    100% |█████████████████| 997/997 [7.2s elapsed, 0s remaining, 73.1 samples/s]       


Image IDs and annotations have been remapped successfully.
Annotations in the second JSON file have been updated successfully.
Filtering annotations...
Annotations have been filtered successfully.
Garbage annotations have been saved successfully.
 100% |█████████████████| 997/997 [8.9s elapsed, 0s remaining, 57.7 samples/s]       


09/04/2024 11:23:24 - INFO - eta.core.utils -    100% |█████████████████| 997/997 [8.9s elapsed, 0s remaining, 57.7 samples/s]       


Image IDs and annotations have been remapped successfully.
Annotations in the second JSON file have been updated successfully.
Filtering annotations...
Annotations have been filtered successfully.
Garbage annotations have been saved successfully.
 100% |█████████████████| 997/997 [11.4s elapsed, 0s remaining, 49.3 samples/s]      


09/04/2024 11:24:25 - INFO - eta.core.utils -    100% |█████████████████| 997/997 [11.4s elapsed, 0s remaining, 49.3 samples/s]      


Image IDs and annotations have been remapped successfully.
Annotations in the second JSON file have been updated successfully.
Filtering annotations...
Annotations have been filtered successfully.
Garbage annotations have been saved successfully.


Evaluation of the model (we use the base fiftyone function)

In [205]:
yolo_results = dataset.evaluate_detections(
    "yolo_predictions_filtered",
    gt_field="annotations",
    eval_key="yolo_eval",
    compute_mAP=True,
)

detr_results = dataset.evaluate_detections(
    "detr_predictions_filtered",
    gt_field="annotations",
    eval_key="detr_eval",
    compute_mAP=True,
)

det2_results = dataset.evaluate_detections(
    "det2_predictions_filtered",
    gt_field="annotations",
    eval_key="det2_eval",
    compute_mAP=True,
)

Evaluating detections...


08/30/2024 10:36:49 - INFO - fiftyone.utils.eval.detection -   Evaluating detections...


 100% |█████████████████| 997/997 [50.0s elapsed, 0s remaining, 6.4 samples/s]       


08/30/2024 10:37:39 - INFO - eta.core.utils -    100% |█████████████████| 997/997 [50.0s elapsed, 0s remaining, 6.4 samples/s]       


Performing IoU sweep...


...

08/30/2024 10:40:31 - INFO - eta.core.utils -    100% |█████████████████| 997/997 [1.0m elapsed, 0s remaining, 4.4 samples/s]        


Performing IoU sweep...


08/30/2024 10:40:31 - INFO - fiftyone.utils.eval.coco -   Performing IoU sweep...


 100% |█████████████████| 997/997 [34.0s elapsed, 0s remaining, 10.7 samples/s]      


08/30/2024 10:41:05 - INFO - eta.core.utils -    100% |█████████████████| 997/997 [34.0s elapsed, 0s remaining, 10.7 samples/s]      


In [206]:
print(" YOLO evaluation on test dataset :")
yolo_results.print_report(classes=dataset.get_classes('annotations'))
print("YOLO mAP :",yolo_results.mAP())

print(" DETR evaluation on test dataset :")
detr_results.print_report(classes=dataset.get_classes('annotations'))
print("DETR mAP :",detr_results.mAP())

print(" Detectron2 evaluation on test dataset :")
det2_results.print_report(classes=dataset.get_classes('annotations'))
print("Detectron2 mAP :",det2_results.mAP())

 YOLO evaluation on test dataset :
                      precision    recall  f1-score   support

              hameau       0.91      0.86      0.89      5083
        moulin_a_eau       0.91      0.62      0.74      1558
             clocher       0.81      0.69      0.74       506
             chateau       0.85      0.54      0.66       567
              abbaye       0.67      0.11      0.18        56
              maison       0.93      0.79      0.85     13337
            chapelle       0.73      0.32      0.44       215
               bourg       0.91      0.69      0.79        62
calvaire_ou_oratoire       0.88      0.13      0.23        52
      gentilhommiere       0.00      0.00      0.00        39
             justice       0.00      0.00      0.00        21
               ville       0.62      0.22      0.33        36
              cabane       0.00      0.00      0.00         0
             prieure       1.00      0.17      0.30        40
       moulin_a_vent       0.44   

### Dataframes
This part can be used to export dataframes from the results (in csv, json or xlsx)

In [ ]:
from utils.benchmarking import fo_result2pd
yolo_df_9 = fo_result2pd(yolo_results,file_path="results/f9/filtered_yolo_results.json",save_json=True)
detr_df_9 = fo_result2pd(detr_results,file_path="results/f9/filtered_detr_results.json",save_json=True)
det2_df_9 = fo_result2pd(det2_results,file_path="results/f9/fildet2_results.json",save_json=True)

## Confusion matrices
Those function takes as inputs the json exported from fiftyone. You may also use a built-in function from fiftyone but i find it hard to read and difficult to export.

In [ ]:
from utils.benchmarking import fo_plot_confusion_matrix
plot = fo_plot_confusion_matrix(yolo_results,normalize=False)

In [ ]:
from utils.benchmarking import  build_confusion_matrix, plot_confusion_matrix

In [157]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from sklearn.metrics import confusion_matrix

def load_coco_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data
def iou(box1, box2):
    """Compute the Intersection over Union (IoU) of two bounding boxes."""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area
    return iou

def build_confusion_matrix(gt_json, pred_json, iou_threshold=0.5):
    gt_data = load_coco_json(gt_json)
    pred_data = load_coco_json(pred_json)
    
    gt_annotations = gt_data['annotations']
    pred_annotations = pred_data['annotations']
    
    categories = {cat['id']: cat['name'] for cat in gt_data['categories']}
    num_categories = len(categories)
    gt_images = {img['id']: img for img in gt_data['images']}
    gt_images_names={img['file_name']: img for img in gt_data['images']}
    pred_images_names={img['file_name']: img for img in pred_data['images']}
    pred_images = {img['id']: img for img in pred_data['images']}
    gt_category_ids = set([cat['id'] for cat in gt_data['categories']])
    # Add the background category
    background_id = num_categories + 1
    categories[background_id] = "background"
    
    # Initialize confusion matrix with an extra row/column for the background
    matrix = np.zeros((num_categories + 1, num_categories + 1))
    
    for gt in gt_annotations:
        matched = False
        gt_image_file = gt_images[gt['image_id']]['file_name'] 
        for pred in pred_annotations:
            pred_image_file = pred_images[pred['image_id']]['file_name']
            if gt_image_file == pred_image_file:
                # get the heigth and width of the image of corresponding name but gt_images as ids for keys

                height,width = gt_images_names[gt_image_file]['height'],gt_images_names[gt_image_file]['width']

                gt_box = gt['bbox']
                # gt_box[0], gt_box[2] = gt_box[0] * width, gt_box[2] * width
                # gt_box[1], gt_box[3] = gt_box[1] * height, gt_box[3] * height
                gt_category_id = gt['category_id']

                pred_box = pred['bbox']
                # pred_box[0], pred_box[2] = pred_box[0] * width, pred_box[2] * width
                # pred_box[1], pred_box[3] = pred_box[1] * height, pred_box[3] * height
                pred_category_id = pred['category_id']
                
                if iou(gt_box, pred_box) >= iou_threshold:
                    matrix[gt_category_id-1][pred_category_id-1] += 1
                    matched = True
                    # print(f'{categories[gt_category_id]} - {categories[pred_category_id]}')
                    break
        
        if not matched:
            matrix[gt_category_id-1][background_id-1] += 1
    
    for pred in pred_annotations:
        matched = False
        pred_image_file = pred_images[pred['image_id']]['file_name']               
        for gt in gt_annotations:
            gt_image_file = gt_images[gt['image_id']]['file_name']
            if gt_image_file == pred_image_file:
                height,width = pred_images_names[pred_image_file]['height'],pred_images_names[pred_image_file]['width']

                pred_box = pred['bbox']
                # pred_box[0], pred_box[2] = pred_box[0] * width, pred_box[2] * width
                # pred_box[1], pred_box[3] = pred_box[1] * height, pred_box[3] * height
                pred_category_id = pred['category_id']

                gt_box = gt['bbox']
                # gt_box[0], gt_box[2] = gt_box[0] * width, gt_box[2] * width
                # gt_box[1], gt_box[3] = gt_box[1] * height, gt_box[3] * height
                gt_category_id = gt['category_id']        

                if iou(gt_box, pred_box) >= iou_threshold:
                    matched = True
                    break
        
        if not matched:
            matrix[background_id-1][pred_category_id-1] += 1
    
    # Sort the categories and reorder the matrix accordingly
    sorted_category_ids = sorted(categories.keys())
    reordered_matrix = np.zeros((len(sorted_category_ids), len(sorted_category_ids)))
    
    for i, gt_id in enumerate(sorted_category_ids):
        for j, pred_id in enumerate(sorted_category_ids):
            reordered_matrix[i, j] = matrix[gt_id-1 if gt_id != background_id else -1, pred_id-1 if pred_id != background_id else -1]
    
    # Update categories to match the sorted order
    sorted_categories = {i+1: categories[sorted_category_ids[i]] for i in range(len(sorted_category_ids))}
    
    return reordered_matrix, sorted_categories

